In [1]:
import argparse
import json
from models import MLPClassifier, Baseline_Embeddings
from models import Seq2Seq, MLP_D, MLP_G, MLP_I, MLP_I_AE, JSDistance, Seq2SeqCAE, Baseline_Embeddings, Baseline_LSTM
from utils import to_gpu, Corpus, batchify, SNLIDataset, collate_snli
import random
import pickle as pkl
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable


In [2]:
#python3.6 train_surrogate.py --data_path ./data/classifier --save_path game_output/ --classifier_path ./data --load_pretrained .
cur_dir = '.'

with open(cur_dir + '/vocab.json', 'r') as fin:
    corpus_vocab = json.load(fin)

corpus_train = SNLIDataset(train=True, vocab_size=11004-4, path='./data/classifier')
corpus_test = SNLIDataset(train=False, vocab_size=11004-4, path='./data/classifier')
trainloader= torch.utils.data.DataLoader(corpus_train, batch_size = 32, collate_fn=collate_snli, shuffle=True)
train_iter = iter(trainloader)
testloader= torch.utils.data.DataLoader(corpus_test, batch_size = 32, collate_fn=collate_snli, shuffle=False)
random.seed(1111)
np.random.seed(1111)
torch.manual_seed(1111)

EPS = 3e-2


original vocab 41574; pruned to 11004
Number of sentences dropped from ./data/classifier/train.txt: 448221 out of 549367 total
original vocab 41574; pruned to 11004
Number of sentences dropped from ./data/classifier/test.txt: 8288 out of 9824 total


In [3]:
autoencoder = torch.load(open(cur_dir + '/models/autoencoder_model.pt', 'rb'))
#gan_gen = torch.load(open(cur_dir + '/models/gan_gen_model.pt', 'rb'))
#gan_disc = torch.load(open(cur_dir + '/models/gan_disc_model.pt', 'rb'))
inverter = torch.load(open(cur_dir + '/models/inverter_model.pt', 'rb'))

# classifier1 = Baseline_Embeddings(100, vocab_size=11004)
classifier1 = Baseline_LSTM(100,300,maxlen=10, gpu=True)
classifier1.load_state_dict(torch.load('./models' + "/baseline/model_lstm.pt"))
classifier1 = classifier1.cuda()
vocab_classifier1 = pkl.load(open('./models' + "/vocab.pkl", 'rb'))

mlp_classifier = MLPClassifier(100 * 2, 3, layers='250-200-100-50')
#if not args.train_mode:

#SURROGATE HAS TO BE TRAINED FROM LSTM IF YOU ARE USING LSTM AS A VICTIM MODEL!!!
mlp_classifier.load_state_dict(torch.load('/home/game/Desktop/natural-adversary/text/surrogate{0}lstm.pt'.format('250-200-100-50')))

print(classifier1)
print(autoencoder)
print(inverter)
print(mlp_classifier)

optimizer = optim.Adam(mlp_classifier.parameters(),
                           lr=1e03,
                           betas=(0.9, 0.999))


/home/game/.local/lib/python3.6/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'models.Seq2SeqCAE' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


LSTM(100, 300, batch_first=True)
Baseline_LSTM(
  (embedding_prem): Embedding(11008, 100)
  (embedding_hypo): Embedding(11008, 100)
  (premise_encoder): LSTM(100, 300, batch_first=True)
  (hypothesis_encoder): LSTM(100, 300, batch_first=True)
  (layers): Sequential(
    (layer1): Linear(in_features=600, out_features=400, bias=True)
    (bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation1): ReLU()
    (layer2): Linear(in_features=400, out_features=100, bias=True)
    (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation2): ReLU()
    (layer3): Linear(in_features=100, out_features=3, bias=True)
    (softmax): Softmax()
  )
)
Seq2SeqCAE(
  (embedding): Embedding(11004, 300)
  (embedding_decoder): Embedding(11004, 300)
  (encoder): Sequential(
    (layer-1): Conv1d(300, 500, kernel_size=(3,), stride=(1,))
    (bn-1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [4]:
from torch.autograd import Variable

def evaluate_model():
    classifier1.eval()

    test_iter = iter(trainloader)
    correct=0
    total=0
    for batch in test_iter:
        premise, hypothesis, target, _, _, _, _ = batch

        if args.cuda:
            premise=premise.cuda()
            hypothesis = hypothesis.cuda()
            target = target.cuda()

        prob_distrib = classifier1.forward((premise, hypothesis))
        predictions = np.argmax(prob_distrib.data.cpu().numpy(), 1)
        correct+=len(np.where(target.data.cpu().numpy()==predictions)[0])
        total+=premise.size(0)
    acc=correct/float(total)
    print("Accuracy:{0}".format(acc))
    return acc

autoencoder.gpu = True
autoencoder = autoencoder.cuda()
autoencoder.start_symbols = autoencoder.start_symbols.cuda()
#gan_gen = gan_gen.cuda()
#gan_disc = gan_disc.cuda()
classifier1 = classifier1.cuda()
inverter = inverter.cuda()
mlp_classifier = mlp_classifier.cuda()

In [5]:
def train_process(premise, hypothesis, target, premise_words, hypothesis_words, premise_length, hypothesis_length):
    #mx = target.max().item()
    #assert(mx >= 0 and mx < 3)
    #for s, s_w in zip(premise, premise_words):
    #    for i, w in zip(s, s_w):
    #        assert(corpus_vocab.get(w, 3) == i)
    #print(hypothesis_words, flush=True)
    autoencoder.eval()
    inverter.eval()
    classifier1.eval()
    mlp_classifier.train()

    #print(premise.max().item(), flush=True)
    #print(hypothesis.max().item(), flush=True)

    premise_idx = torch.tensor([[corpus_vocab.get(w, 3) for w in s] for s in premise_words]).cuda()
    hypothesis_idx = torch.tensor([[corpus_vocab.get(w, 3) for w in s] for s in hypothesis_words]).cuda()

    c_prem = autoencoder.encode(premise_idx, premise_length, noise=False)
    z_prem = inverter(c_prem).detach()

    c_hypo = autoencoder.encode(hypothesis_idx, hypothesis_length, noise=False)
    z_hypo = inverter(c_hypo).detach()

    # z_comb = nn.cat((z_prem, z_hypo), 0).detach()

    output = mlp_classifier(z_prem, z_hypo)
    gold = classifier1((premise, hypothesis)).detach()

    #print(output.shape, flush=True)
    #print(gold.shape, flush=True)

    acc = (torch.argmax(gold, 1) == target).to(torch.float32).mean().item()
    acc_surrogate = (torch.argmax(output, 1) == target).to(torch.float32).mean().item()


    loss = -torch.mean(torch.sum(output * F.softmax(gold, dim=1), 1), 0)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item(), acc, acc_surrogate


In [6]:
def classifier_pred(pw, hw):
    classifier1.eval()

    premise_idx = torch.tensor([vocab_classifier1.get(w, 3) for w in pw]).cuda().unsqueeze(0)
    hypothesis_idx = torch.tensor([vocab_classifier1.get(w, 3) for w in hw]).cuda().unsqueeze(0)

    return F.softmax(classifier1((premise_idx, hypothesis_idx)), 1).squeeze(0).cpu().detach().numpy()


In [7]:
def cross_entropy(p, q):
    q = torch.log(q)
    a = p * q
    a = torch.sum(a)
    a = -a
    return a

In [30]:
ALPHA = 0.05

In [33]:
def perturb(criterion, premise, hypothesis, target, premise_words, hypothesis_words, premise_length, hypothesis_length):
    autoencoder.eval()
    inverter.eval()
    classifier1.eval()
    mlp_classifier.eval()

    premise_words = [premise_words]
    hypothesis_words = [hypothesis_words]
    premisea_length = [premise_length]
    hypothesis_length = [hypothesis_length]


    premise_idx = torch.tensor([[corpus_vocab.get(w, 3) for w in s] for s in premise_words]).cuda()
    hypothesis_idx = torch.tensor([[corpus_vocab.get(w, 3) for w in s] for s in hypothesis_words]).cuda()

    c_prem = autoencoder.encode(premise_idx, premise_length, noise=False)
    z_prem = inverter(c_prem).detach()

#     c_hypo = autoencoder.encode(hypothesis_idx, hypothesis_length, noise=False).detach()
#     c_hypo.requires_grad = True
    c_hypo = autoencoder.encode(hypothesis_idx, hypothesis_length, noise=False)
    z_hypo = inverter(c_hypo)
    
    premise = premise.unsqueeze(0)
    hypothesis = hypothesis.unsqueeze(0)
    target = target.unsqueeze(0)

    mlp_classifier.zero_grad()
    inverter.zero_grad()
    
    #temp = torch.Tensor(1, 300).fill_(0).cuda().detach()
    for j in range(1):
        temp = torch.Tensor(1, 300).normal_(0, 0.1).cuda().detach()
        temp.requires_grad=True
        c_hypoprime = [{'params': temp}]
        optimizer = torch.optim.Adam(c_hypoprime, lr=1e-4)

        for i in range(2000):
            output2 = torch.nn.functional.softmax(classifier1.forward((premise_idx, hypothesis_idx))).detach()
            z_hypoprime = inverter(c_hypoprime[0]['params'][0])
            output3 = torch.nn.functional.softmax(mlp_classifier(z_prem, z_hypoprime))
            loss4 = kl_divergence(output3, output2) + ALPHA * torch.norm(z_hypoprime, p=2)
            optimizer.zero_grad()
            loss4.backward()
            print(loss4)
            optimizer.step()
        if(j == 0):
            bestloss = loss4
            bestadv = c_hypoprime
        elif(bestloss > loss4):
            bestloss = loss4
            bestadv = c_hypoprime        
            
    c_hypoprime = bestadv
    nhypo_idx = autoencoder.generate(c_hypoprime[0]['params'][0], 10, False)
    return nhypo_idx.squeeze(0).cpu().numpy()

In [43]:
def perturb_premiseonly(criterion, premise, hypothesis, target, premise_words, hypothesis_words, premise_length, hypothesis_length):
    autoencoder.eval()
    inverter.eval()
    classifier1.eval()
    mlp_classifier.eval()

    premise_words = [premise_words]
    hypothesis_words = [hypothesis_words]
    premise_length = [premise_length]
    hypothesis_length = [hypothesis_length]


    premise_idx = torch.tensor([[corpus_vocab.get(w, 3) for w in s] for s in premise_words]).cuda()
    hypothesis_idx = torch.tensor([[corpus_vocab.get(w, 3) for w in s] for s in hypothesis_words]).cuda()

    c_prem = autoencoder.encode(premise_idx, premise_length, noise=False)
    z_prem = inverter(c_prem).detach()

#     c_hypo = autoencoder.encode(hypothesis_idx, hypothesis_length, noise=False).detach()
#     c_hypo.requires_grad = True
#     c_hypo = autoencoder.encode(hypothesis_idx, hypothesis_length, noise=False)
#     z_hypo = inverter(c_hypo)
    
    premise = premise.unsqueeze(0)
    hypothesis = hypothesis.unsqueeze(0)
    target = target.unsqueeze(0)

#     mlp_classifier.zero_grad()
#     inverter.zero_grad()
    for j in range(1):
        temp = torch.Tensor(1, 300).normal_(0, 0.1).cuda().detach()
        temp.requires_grad=True
        c_hypoprime = [{'params': temp}]
        optimizer = torch.optim.Adam(c_hypoprime, lr=1e-4)
        for i in range(500):
            objtive = torch.tensor([[0.98, 0.01, 0.01]]).cuda()
            hypothesis_idx = autoencoder.generate(c_hypoprime[0]['params'][0], 10, False)
            z_hypoprime = inverter(c_hypoprime[0]['params'][0])
            output2 = torch.nn.functional.softmax(mlp_classifier(z_prem, z_hypoprime))
            loss4 = cross_entropy(output2, objtive)
            optimizer.zero_grad()
            loss4.backward()
            optimizer.step()
        if(j == 0):
            bestloss = loss4
            bestadv = c_hypoprime
        elif(bestloss > loss4):
            bestloss = loss4
            bestadv = c_hypoprime        
            
    c_hypoprime = bestadv
    nhypo_idx = autoencoder.generate(c_hypoprime[0]['params'][0], 10, False)
    return nhypo_idx.squeeze(0).cpu().numpy()

In [10]:
def maximum(array):
    a = array[0]
    idx = 0
    for i in range(1, 3):
        if(a < array[i]):
            idx = i
            a = array[i]
    return idx

In [11]:
def kl_divergence(p, q):
    k = torch.log(p / q)
    p = p * k
    return torch.sum(p)

In [12]:
import secrets

def samples(alist):
    secure_random = secrets.SystemRandom()
    num_to_select = int(len(alist) / 2)
    list_of_random_items = secure_random.sample(alist, num_to_select)
    return list_of_random_items

In [44]:
criterion = nn.CrossEntropyLoss().cuda()

niter = 0

idx2words = dict(map(lambda x: (x[1], x[0]), corpus_vocab.items()))
oldcorrect = 0
newcorrect = 0
n = 0
alloutputarr = []
while niter < len(testloader):
    niter += 1
    batch = train_iter.next()
    outputarr = []
    for p, h, t, pw, hw, pl, hl in zip(*batch):
        nh = perturb_premiseonly(criterion, p.cuda(), h.cuda(), t.cuda(), pw, hw, pl, hl)
        print('--------------------------------')
        print('Target ', t)
        print(' '.join(pw))
        print(' '.join(hw))
#         outputarr.append(t)
#         outputarr.append(' '.join(pw))
#         outputarr.append(' '.join(hw))
        nhw = (['<sos>'] + [idx2words[i] for i in nh])[:10]
        print(' '.join(nhw))
        oldpred = classifier_pred(pw, hw)
        newpred = classifier_pred(pw, nhw)
        print('Old ', oldpred)
        print('New ', newpred)
        print('Old Prediction: ' + str(maximum(oldpred)))
        print('New Prediction: ' + str(maximum(newpred)))
        print('similarity: ' + str(kl_divergence(torch.tensor(newpred), torch.tensor(oldpred))))
        if(maximum(oldpred) == t.item()):
            oldcorrect = oldcorrect + 1
        if(maximum(newpred) == t.item()):
            newcorrect = newcorrect + 1
        n = n + 1
#         outputarr.append(' '.join(nhw))
#         outputarr.append(oldpred)
#         outputarr.append(newpred)
#         outputarr.append(maximum(oldpred))
#         outputarr.append(maximum(newpred))
#         outputarr.append(kl_divergence(torch.tensor(newpred), torch.tensor(oldpred)))
#         alloutputarr.append(outputarr)
print('oldcorrect: ' + str(oldcorrect))
print('newcorrect: ' + str(newcorrect))
print('number of premises ' + str(n))

# Target  tensor(0)
# <sos> a man climbs a large rock . <eos> <pad>
# <sos> a man is climbing . <pad> <pad> <pad> <pad>
# <sos> people are standing in a large open area .
# Old  [0.5761169  0.21194157 0.21194157]
# New  [0.5761136  0.2119435  0.21194291]
# Old Prediction: 0
# New Prediction: 0
# similarity: tensor(-4.6381e-08)
#0 entailment #1 neutral #2 contradiction

/home/game/.local/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


--------------------------------
Target  tensor(0)
<sos> a backhoe digs out dirt at dusk . <eos>
<sos> there is some digging going on at dusk .
<sos> a man splashes across water . <eos> <eos> <eos>
Old  [0.57608885 0.21196154 0.2119496 ]
New  [0.54641867 0.21784037 0.23574093]
Old Prediction: 0
New Prediction: 0
similarity: tensor(0.0021)
--------------------------------
Target  tensor(0)
<sos> skateboarder doing a trick at a skate park .
<sos> a skateboarder is at a skate park . <pad>
<sos> a dog attacks behind a harness . <eos> <eos>
Old  [0.57611686 0.21194158 0.21194158]
New  [0.21194156 0.21194156 0.57611686]
Old Prediction: 0
New Prediction: 2
similarity: tensor(0.3642)
--------------------------------
Target  tensor(2)
<sos> two construction workers tying something to a crane .
<sos> a bulldozer is in the shop getting fixed .
<sos> a person is fixing hair . <eos> <eos> <eos>
Old  [0.2120377  0.21207164 0.57589066]
New  [0.48805386 0.28299353 0.22895263]
Old Prediction: 2
New Pre

--------------------------------
Target  tensor(0)
<sos> a child in pajamas makes cookies <eos> <pad> <pad>
<sos> a child makes cookies . <pad> <pad> <pad> <pad>
<sos> bikers talk along wearing a wave . <eos> <eos>
Old  [0.57611626 0.21194169 0.21194203]
New  [0.5547812  0.2169271  0.22829176]
Old Prediction: 0
New Prediction: 0
similarity: tensor(0.0011)
--------------------------------
Target  tensor(2)
<sos> two men holding skateboards and one man skateboarding .
<sos> three men are watching the half pipe competition .
<sos> a man is a priest . <eos> <eos> <eos>
Old  [0.23297133 0.43508032 0.3319483 ]
New  [0.21221247 0.21281303 0.57497454]
Old Prediction: 1
New Prediction: 2
similarity: tensor(0.1439)
--------------------------------
Target  tensor(0)
<sos> two people sitting out looking at the water .
<sos> the two people were sitting watching the water <pad>
<sos> two short haired kids bike . <eos> <eos> <eos>
Old  [0.45912194 0.23019476 0.3106833 ]
New  [0.21339163 0.21598057 0.

--------------------------------
Target  tensor(0)
<sos> two persons repairing their bicycle . <eos> <pad> <pad>
<sos> there are two people repairing something . <pad> <pad>
<sos> man is knitting a boxing in line . <eos>
Old  [0.5761169  0.21194157 0.21194157]
New  [0.21194217 0.21194369 0.5761142 ]
Old Prediction: 0
New Prediction: 2
similarity: tensor(0.3642)
--------------------------------
Target  tensor(2)
<sos> a woman sits in a subway station . <eos>
<sos> the woman is standing at an airport terminal .
<sos> someone is having fun . <eos> <eos> <eos> birthday
Old  [0.2119418  0.21194202 0.5761162 ]
New  [0.21195443 0.57606983 0.21197571]
Old Prediction: 2
New Prediction: 1
similarity: tensor(0.3641)
--------------------------------
Target  tensor(1)
<sos> a small child posing with goofy the dog .
<sos> the child is at disney world . <pad> <pad>
<sos> a chef carrying something down . <eos> <eos> <eos>
Old  [0.21773386 0.5665039  0.21576233]
New  [0.5745122  0.21305162 0.21243623]


--------------------------------
Target  tensor(2)
<sos> four road workers planting a tree . <eos> <pad>
<sos> there is no one outside . <pad> <pad> <pad>
<sos> a toddler is waiting being read . <eos> <eos>
Old  [0.21194158 0.21194158 0.57611686]
New  [0.5761164  0.21194172 0.21194185]
Old Prediction: 2
New Prediction: 0
similarity: tensor(0.3642)
--------------------------------
Target  tensor(0)
<sos> the animals look very happy and healthy . <eos>
<sos> the animals are in excellent health . <pad> <pad>
<sos> the has a picture of nobody <eos> . <eos>
Old  [0.5761166  0.21194163 0.21194173]
New  [0.21202175 0.21216945 0.57580876]
Old Prediction: 0
New Prediction: 2
similarity: tensor(0.3638)
--------------------------------
Target  tensor(2)
<sos> five boys are buried in sand <oov> . <eos>
<sos> two girls playing <pad> <pad> <pad> <pad> <pad> <pad>
<sos> people <oov> is playing sports <eos> <eos> <eos> <eos>
Old  [0.57611686 0.21194158 0.2119416 ]
New  [0.21199103 0.57594985 0.2120591

...

--------------------------------
Target  tensor(0)
<sos> children are playing on a playground . <eos> <pad>
<sos> the children are outside . <pad> <pad> <pad> <pad>
<sos> two ladies dancing over wilson . <eos> <eos> woman
Old  [0.5761169  0.21194157 0.21194157]
New  [0.22920154 0.303304   0.46749443]
Old Prediction: 0
New Prediction: 2
similarity: tensor(0.2673)
--------------------------------
Target  tensor(0)
<sos> two women on a subway talking while standing .
<sos> two women on the subway are talking while standing
<sos> man he is flipping . <eos> <eos> <eos> dogs
Old  [0.21203835 0.5757981  0.21216351]
New  [0.2122372  0.57473844 0.21302436]
Old Prediction: 1
New Prediction: 1
similarity: tensor(2.8883e-06)
--------------------------------
Target  tensor(1)
<sos> a bride and groom enjoy the reception . <eos>
<sos> the couple is white <pad> <pad> <pad> <pad> <pad>
<sos> several people on ride atvs . <eos> <eos> .
Old  [0.57611585 0.21194227 0.21194188]
New  [0.2227584  0.21473263 

--------------------------------
Target  tensor(2)
<sos> a little girl playing with kitchen toys . <eos>
<sos> the little girl is finger painting . <pad> <pad>
<sos> there are holding men on a bicycle . <eos>
Old  [0.5761166  0.21194163 0.21194181]
New  [0.5760867 0.211948  0.2119653]
Old Prediction: 0
New Prediction: 0
similarity: tensor(-4.3556e-08)
--------------------------------
Target  tensor(0)
<sos> ladies looking in a photo album . <eos> <pad>
<sos> the ladies are looking at something . <pad> <pad>
<sos> people cheer down in a parade . <eos> <eos>
Old  [0.5761169  0.21194157 0.21194157]
New  [0.54408073 0.22027855 0.23564073]
Old Prediction: 0
New Prediction: 0
similarity: tensor(0.0023)
--------------------------------
Target  tensor(2)
<sos> a man and woman sitting near the street .
<sos> a male and female are playing tennis . <pad>
<sos> child are under a naked <eos> <eos> <eos> kids
Old  [0.2121888  0.5753143  0.21249695]
New  [0.2222692 0.5190181 0.2587127]
Old Prediction

--------------------------------
Target  tensor(1)
<sos> person crossing a bridge in dim light . <eos>
<sos> a person hits their leg while crossing <pad> <pad>
<sos> two huge dogs are indoors . <eos> <eos> <eos>
Old  [0.5726938  0.21407406 0.21323207]
New  [0.21194711 0.21196216 0.57609075]
Old Prediction: 0
New Prediction: 2
similarity: tensor(0.3598)
--------------------------------
Target  tensor(1)
<sos> three men sit outside a cafe . <eos> <pad>
<sos> three men drink coffee outside a cafe . <pad>
<sos> a contest event are outdoors with people <oov> .
Old  [0.21726169 0.21342899 0.5693093 ]
New  [0.21214335 0.57585454 0.21200208]
Old Prediction: 2
New Prediction: 1
similarity: tensor(0.3571)
--------------------------------
Target  tensor(2)
<sos> a woman with a helmet riding a bike .
<sos> the woman is reading at the beach . <pad>
<sos> there are tents set and a snowy path <eos>
Old  [0.2119624  0.21195015 0.5760874 ]
New  [0.57606566 0.21198198 0.21195234]
Old Prediction: 2
New P

--------------------------------
Target  tensor(2)
<sos> a male student napping on university benches . <eos>
<sos> a male student running <pad> <pad> <pad> <pad> <pad>
<sos> there were watching a blond match <eos> <eos> <eos>
Old  [0.576115   0.21194196 0.21194305]
New  [0.57602626 0.21197955 0.21199419]
Old Prediction: 0
New Prediction: 0
similarity: tensor(-1.7793e-08)
--------------------------------
Target  tensor(1)
<sos> four laughing women sitting around a dining table .
<sos> four friends and coworkers are enjoying dinner together .
<sos> a children paint a buildings and dress . <eos>
Old  [0.21195383 0.5760655  0.21198073]
New  [0.5752739  0.2121748  0.21255128]
Old Prediction: 1
New Prediction: 0
similarity: tensor(0.3630)
--------------------------------
Target  tensor(0)
<sos> a female artist sketching a portrait . <eos> <pad>
<sos> the female artist is sketching a portrait <pad> <pad>
<sos> people look through a sale . <eos> <eos> kids
Old  [0.22028922 0.24700032 0.532710

--------------------------------
Target  tensor(0)
<sos> someones getting their nails painted red . <eos> <pad>
<sos> nail color will be applied . <pad> <pad> <pad>
<sos> someone skiing on a cold <eos> <eos> <eos> dress
Old  [0.5761049  0.2119441  0.21195102]
New  [0.4740641  0.2298345  0.29610145]
Old Prediction: 0
New Prediction: 0
similarity: tensor(0.0252)
--------------------------------
Target  tensor(2)
<sos> a young man sits with a dog outside .
<sos> cat jumping outside with young man <pad> <pad> <pad>
<sos> a tired and dog are many <eos> <eos> <eos>
Old  [0.2160816  0.21303107 0.5708873 ]
New  [0.21194166 0.57611674 0.21194166]
Old Prediction: 2
New Prediction: 1
similarity: tensor(0.3591)
--------------------------------
Target  tensor(1)
<sos> a woman walking with a backpack . <eos> <pad>
<sos> a woman is walking to school . <pad> <pad>
<sos> a man is catching sports car . <eos> <eos>
Old  [0.43772584 0.23459534 0.32767886]
New  [0.21194169 0.21194172 0.5761166 ]
Old Predic